In [1]:
import os
import requests
import pandas as pd
from mysql import connector
from dotenv import load_dotenv
import numpy as np
from datetime import datetime

In [2]:
load_dotenv()

url_base = "https://api.coingecko.com/api/v3/exchanges"
params = {
    "per_page": 50,
    "page": 1
}

print(params)

{'per_page': 50, 'page': 1}


# EXTRACT

In [6]:
response = requests.get(url_base, params=params)
payload = response.json()
payload[0]

{'id': 'binance',
 'name': 'Binance',
 'year_established': 2017,
 'country': 'Cayman Islands',
 'description': 'One of the world’s largest cryptocurrency exchanges by trading volume, offering a wide range of services including spot, futures, and staking options.',
 'url': 'https://www.binance.com/',
 'image': 'https://coin-images.coingecko.com/markets/images/52/small/binance.jpg?1706864274',
 'has_trading_incentive': False,
 'trust_score': 10,
 'trust_score_rank': 1,
 'trade_volume_24h_btc': 161473.2248231604}

# TRANSFORM

In [167]:
exchanges_df = pd.DataFrame(payload)
exchanges_df.head()

,id,name,year_established,country,description,url,image,has_trading_incentive,trust_score,trust_score_rank,trade_volume_24h_btc
0,binance,Binance,2017,Cayman Islands,One of the world’s largest cryptocurrency exch...,https://www.binance.com/,https://coin-images.coingecko.com/markets/imag...,False,10,1,108174.298360
1,bybit_spot,Bybit,2018,British Virgin Islands,Bybit is the world’s second-largest cryptocurr...,https://www.bybit.com,https://coin-images.coingecko.com/markets/imag...,False,10,2,23308.164569
2,mxc,MEXC,2018,Seychelles,"Established in April 2018, MEXC is one of the ...",https://www.mexc.com/,https://coin-images.coingecko.com/markets/imag...,False,10,3,23206.970726
3,gate,Gate,2013,Panama,"Established in 2013, Gate is a cryptocurrency ...",https://www.gate.com,https://coin-images.coingecko.com/markets/imag...,False,10,4,21699.157418
4,bitget,Bitget,2018,Seychelles,"Established in 2018, Bitget is the world's lea...",https://www.bitget.com/,https://coin-images.coingecko.com/markets/imag...,False,10,5,19603.058720


In [168]:
exchanges_df = exchanges_df.drop(columns=['has_trading_incentive', 'description'], errors='ignore')
exchanges_df = exchanges_df.round(5)

exchanges_df.rename(columns={"id": "exchange_id", "image": "image_url"}, inplace=True)

exchanges_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 9 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   exchange_id           50 non-null     object 
 1   name                  50 non-null     object 
 2   year_established      50 non-null     int64  
 3   country               49 non-null     object 
 4   url                   50 non-null     object 
 5   image_url             50 non-null     object 
 6   trust_score           50 non-null     int64  
 7   trust_score_rank      50 non-null     int64  
 8   trade_volume_24h_btc  50 non-null     float64
dtypes: float64(1), int64(3), object(5)
memory usage: 3.6+ KB


# LOAD

In [169]:
MYSQL_USER = os.getenv("MYSQL_USER")
MYSQL_PASSWORD = os.getenv("MYSQL_PASSWORD")
MYSQL_HOST = os.getenv("MYSQL_HOST")
MYSQL_PORT = os.getenv("MYSQL_PORT")
MYSQL_DB = os.getenv("MYSQL_DATABASE")

In [170]:
db_conn = connector.connect(
    host=MYSQL_HOST,
    user=MYSQL_USER,
    password=MYSQL_PASSWORD,
    port=MYSQL_PORT,
    database=MYSQL_DB,
    connection_timeout=10,
    autocommit=False,
    raise_on_warnings=True
)   

db_cur = db_conn.cursor()
print(f"[SUCCESS] Connected to MySQL db {MYSQL_HOST}:{MYSQL_PORT}/{MYSQL_DB} as user {MYSQL_USER}")

[SUCCESS] Connected to MySQL db localhost:3306/cryptodb as user root


In [171]:
sql_table = "exchanges"
db_cur.execute(f"SHOW TABLES LIKE '{sql_table}'")

if db_cur.fetchone() is None:
    raise SystemExit(f"[ERROR] Table '{sql_table}' does not exist in database '{MYSQL_DB}'")
else:
    print(f"[SUCCESS] Table '{sql_table}' exists in database '{MYSQL_DB}'")

[SUCCESS] Table 'exchanges' exists in database 'cryptodb'


In [172]:
UPSERT_SQL = f"""
INSERT INTO {sql_table} (
    exchange_id, name, year_established, country, url, 
    image_url, trust_score, trust_score_rank, trade_volume_24h_btc
)
VALUES (
    %s, %s, %s, %s, %s,
    %s, %s, %s, %s
) AS src
ON DUPLICATE KEY UPDATE
    exchange_id = src.exchange_id,
    name = src.name, 
    year_established = src.year_established,
    country = src.country,
    url = src.url, 
    image_url = src.image_url,
    trust_score = src.trust_score, 
    trust_score_rank = src. trust_score_rank, 
    trade_volume_24h_btc = src.trade_volume_24h_btc
"""

In [173]:
exchanges_list = exchanges_df.values.tolist()

try:
    db_cur.executemany(UPSERT_SQL, exchanges_list)
    db_conn.commit()
    print(f"[SUCCESS] Upserted {db_cur.rowcount} records into table '{sql_table}'")
except connector.Error as err:
    db_conn.rollback()
    print(f"[ERROR] Failed to upsert records into table '{sql_table}': {err}")
finally:
    db_cur.close()
    db_conn.close()
    print("[INFO] MySQL connection closed")

[SUCCESS] Upserted 99 records into table 'exchanges'
[INFO] MySQL connection closed
